In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import os, json
from tqdm import tqdm
import torch as ch
from torchvision import models, transforms
from torch.autograd import Variable

from lime import lime_image
from skimage.segmentation import mark_boundaries

In [2]:
import utils
import implem_utils
from scorecam import ScoreCam

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

def get_input_tensors(img):
    # unsqeeze converts single image to batch of 1
    return transform(img).unsqueeze(0)

In [4]:
def get_model(path):
    model = utils.FaceModel(512,
                            train_feat=True,
                            weight_init=None,
                            hidden=[64, 16]).cuda()
    model = nn.DataParallel(model)
    model.load_state_dict(ch.load(path), strict=False)
    model.eval()
    return model

In [12]:
constants = utils.Celeb()
ds = constants.get_dataset()

attrs = constants.attr_names

In [5]:
def raw_255_image(z):
    z_ = z.numpy().transpose(1, 2, 0)
    z_ = 0.5 * z_ + 0.5
    return (z_ * 255).astype(np.uint8)

In [6]:
def show_image(z):
    plt.imshow(raw_255_image(z))

In [7]:
def batch_predict(model, images):
    batch = ch.stack(tuple(transform(i) for i in images), dim=0)
    
    logits = model(batch.cuda()).detach()
    probs = ch.sigmoid(logits)
    probs = ch.stack([1 - probs[:, 0], probs[:, 0]], 1)
    return probs.cpu().numpy()

In [8]:
def get_scores(model):
    constants = utils.Celeb()
    ds = constants.get_dataset()
    td = utils.CelebACustomBinary(
        "/p/adversarialml/as9rw/datasets/celeba_raw_crop/splits/70_30/all/split_2/test",
        transform=transform)
    cropped_dataloader = DataLoader(td,
                                batch_size=15,
                                shuffle=True)
    
    explainer = lime_image.LimeImageExplainer()
    
    def model_batch_predict(x):
        return batch_predict(model, x)

    scores = []
    labels = []

    for i, (x, y) in tqdm(enumerate(cropped_dataloader)):
        x_raw = [raw_255_image(x_) for x_ in x]
        labels.append(y.numpy())
        
        if i < 1:
            continue
    
        for img_t in x_raw:
            explanation = explainer.explain_instance(img_t,
                                                     model_batch_predict, # classification function
                                                     top_labels=1, 
                                                     hide_color=0, 
                                                     num_samples=2000)
    
            temp, mask = explanation.get_image_and_mask(explanation.top_labels[0],
                                                        positive_only=True,
                                                        num_features=5,
                                                        hide_rest=False)
    
            img_boundry1 = mark_boundaries(temp/255.0, mask)
            plt.imshow(img_boundry1)
            return (temp, mask)
            
            temp_pos = temp * (np.expand_dims(mask, 2))
            temp_neg = temp * (1 - np.expand_dims(mask, 2))
            test_pred = model_batch_predict([temp_pos.astype(np.uint8), temp_neg.astype(np.uint8)])
            scores.append(test_pred[:, 1])
        
        if i == 3: break
    
    return np.stack(scores, 0), np.concatenate(labels, 0)

In [9]:
model = get_model("/u/as9rw/work/fnb/implems/celeba_models_split/70_30/split_1/all/augment_vggface/20_0.9151053864168618.pth")

In [ ]:
temp, mask = get_scores(model)

In [ ]:
temp_ = ch.unsqueeze(ch.from_numpy(temp / 255.), 0)
temp_ = temp_.permute(0, 3, 1, 2).float().cuda()
score_cam = ScoreCam(model, target_layer=11)
# Generate cam mask
cam = score_cam.generate_cam(temp_, target_class=0)

In [ ]:
import matplotlib.cm as mpl_color_map

# color_map = mpl_color_map.get_cmap('hsv')
# no_trans_heatmap = color_map(cam)
tempo = temp.copy()

cam_ = (cam >= 0.4)

for i in range(3):
    tempo[:, :, i] = (cam_ * tempo[:, :, i]).astype(np.uint8)

plt.imshow(tempo)

In [ ]:
def mask_auto_complete(model, image, mask, target, lr):
    loss_fn = nn.BCEWithLogitsLoss()
    image_ = image/255
    x_opt = ch.from_numpy(image_).cuda()
    x_opt = x_opt.permute(2, 0, 1).float()

    mask_ = ch.from_numpy(1 - mask).cuda()
    mask_ = ch.unsqueeze(ch.unsqueeze(mask_, 0), 0)
    mask_ = mask_.repeat(1, 3, 1, 1)
    
    gt = ch.tensor([[target]]).cuda()
    
    inp = ch.unsqueeze((x_opt - 0.5)/0.5, 0).clone() * (1 - mask_)
#     inp += (mask_) * 0.5
    
    # Fill balck with average color of image?
    inp = Variable(inp, requires_grad=True)
    
    iterator = tqdm(range(50))
    for i in iterator:
        logit = model(inp)
        
        loss = loss_fn(logit, gt)
        loss.backward(ch.ones_like(loss), retain_graph=True)
        
        # Back-flow of gradient
        # But only where mask permits it
        inp.data -= (lr * inp.grad * mask_)
        
        # Clamp back image into (-0.5, 0.5)
        inp.data = ch.clamp(inp.data, -0.5, 0.5)
        
        iterator.set_description("Loss: %.4f" % loss.item())
    
    return inp.detach().cpu().clone()

In [ ]:
temp_autofill = mask_auto_complete(model, temp, cam_ * 0,
                                   target=1.,
                                   lr=0.01)

In [ ]:
show_image(temp_autofill[0])

In [32]:
def perf_drop(m, blr=0.25):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5), (0.5))])
    td = utils.CelebACustomBinary(
        "/p/adversarialml/as9rw/datasets/celeba_raw_crop/splits/70_30/all/split_2/test",
        transform=transform)
    cropped_dataloader = DataLoader(td,
                                batch_size=75,
                                shuffle=True)
    correct, total = 0, 0
    for (x, y) in tqdm(cropped_dataloader):
        # Blank our lower blr% of image
        x[:, :, :, -int(blr * x.shape[3]):] = -0.5
        x, y = x.cuda(), y.cuda()
        preds = (model(x)[:, 0] >= 0)
        correct += ch.sum(preds == y[:, attrs.index("Smiling")]).sum().cpu().item()
        total += y.shape[0]
    return correct/total

In [35]:
ratios = [0.25, 0.3, 0.4, 0.5, 0.7, 0.9]
perfs = []
for ratio in ratios:
    perfs.append(perf_drop(model, blr=ratio))

print(perfs)

100%|██████████| 80/80 [00:23<00:00,  3.40it/s]

[0.8966212808875441, 0.8840141200201714, 0.8347621448983022, 0.7528996469994957, 0.4989073793914944, 0.49840309295679946]


In [36]:
model = get_model("/u/as9rw/work/fnb/implems/celeba_models_split/70_30/split_1/male/augment_vggface/20_0.9246347941567065.pth")

ratios = [0.25, 0.3, 0.4, 0.5, 0.7, 0.9]
perfs = []
for ratio in ratios:
    perfs.append(perf_drop(model, blr=ratio))

print(perfs)

100%|██████████| 80/80 [00:22<00:00,  3.55it/s]

[0.9088922507984535, 0.9009917633215666, 0.8473693057656749, 0.7406286770885863, 0.4987392839132627, 0.49840309295679946]


In [ ]:
temp_autofill = mask_auto_complete(model, temp, 1 - mask,
                                   target=1.,
                                   lr=0.0001)

In [ ]:
model = get_model("/u/as9rw/work/fnb/implems/celeba_models_split/70_30/split_1/all/augment_vggface/10_0.928498243559719.pth")
_ = get_scores(model)

In [ ]:
get_scores(model)

In [ ]:
model = get_model("/u/as9rw/work/fnb/implems/celeba_models_split/70_30/split_1/all/augment_vggface/4_0.9207406323185011.pth")
get_scores(model)

In [ ]:
model = get_model("/u/as9rw/work/fnb/implems/celeba_models_split/70_30/split_1/male/augment_vggface/20_0.9246347941567065.pth")
get_scores(model)

In [ ]:
labels.shape

In [ ]:
where_male = np.nonzero(labels[:, attrs.index("Male")])[0]

In [ ]:
where_female = np.nonzero(1 - labels[:, attrs.index("Male")])[0]

In [ ]:
np.mean(scores[where_male, 0] - scores[where_male, 1])

In [ ]:
np.mean(scores[where_female, 0] - scores[where_female, 1])